## Imports

In [1]:
import argparse
import os
import pathlib
import sys

# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io as io
import tifffile
from moviepy.editor import VideoFileClip
from napari.utils import nbscreenshot
from napari_animation import Animation
from napari_animation.easing import Easing
from nviz.image import image_set_to_arrays
from nviz.image_meta import extract_z_slice_number_from_filename, generate_ome_xml
from nviz.view import view_ometiff_with_napari
from PIL import Image

cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd
else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break
sys.path.append(str(root_dir / "utils"))
from arg_parsing_utils import check_for_missing_args, parse_args
from notebook_init_utils import bandicoot_check, init_notebook

root_dir, in_notebook = init_notebook()

image_base_dir = bandicoot_check(
    pathlib.Path(os.path.expanduser("~/mnt/bandicoot")).resolve(), root_dir
)

sys.path.append(f"{root_dir}/utils")
from segmentation_decoupling import euclidian_2D_distance

In [2]:
def mp4_to_gif(input_mp4, output_gif, fps=10):
    clip = VideoFileClip(input_mp4)
    clip = clip.set_fps(fps)  # Reduce FPS to control file size
    clip.write_gif(output_gif, loop=0)  # loop=0 makes it loop forever
    print(f"Converted {input_mp4} to {output_gif}")

In [3]:
def animate_view(
    viewer, output_path_name: str, steps: int = 30, easing: str = "linear", dim: int = 3
):
    animation = Animation(viewer)
    if easing == "linear":
        ease_style = Easing.LINEAR
    else:
        raise ValueError(f"Invalid easing style: {easing}")

    viewer.dims.ndisplay = dim
    # rotate around the y-axis
    viewer.camera.angles = (0.0, 0.0, 90.0)  # (z, y, x) axis of rotation
    animation.capture_keyframe(steps=steps, ease=ease_style)

    viewer.camera.angles = (0.0, 180.0, 90.0)
    animation.capture_keyframe(steps=steps, ease=ease_style)

    viewer.camera.angles = (0.0, 360.0, 90.0)
    animation.capture_keyframe(steps=steps, ease=ease_style)

    viewer.camera.angles = (0.0, 0.0, 270.0)
    animation.capture_keyframe(steps=steps, ease=ease_style)

    viewer.camera.angles = (0.0, 0.0, 90.0)
    animation.capture_keyframe(steps=steps, ease=ease_style)

    animation.animate(output_path_name, canvas_only=True)

In [4]:
if not in_notebook:
    args = parse_args()
    well_fov = args["well_fov"]
    patient = args["patient"]
    check_for_missing_args(
        well_fov=well_fov,
        patient=patient,
    )
else:
    print("Running in a notebook")
    patient = "SARCO361_T1"
    well_fov = "D8-4"

ANIMATE = True

image_dir = pathlib.Path(
    f"{image_base_dir}/data/{patient}/zstack_images/{well_fov}/"
).resolve(strict=True)
label_dir = pathlib.Path(
    f"{image_base_dir}/data/{patient}/segmentation_masks/{well_fov}/"
).resolve(strict=True)
animation_subparent_name = pathlib.Path(
    f"{root_dir}/2.segment_images/animations/{patient}/{well_fov}/"
)
animation_subparent_name.mkdir(parents=True, exist_ok=True)

Running in a notebook


In [5]:
output_path = "output.zarr"
channel_map = {
    "405": "Nuclei",
    "488": "Endoplasmic Reticulum",
    "555": "Actin, Golgi, and plasma membrane (AGP)",
    "640": "Mitochondria",
    "TRANS": "Brightfield",
}
scaling_values = [1, 0.1, 0.1]

In [6]:
frame_zstacks = image_set_to_arrays(
    image_dir,
    label_dir,
    channel_map=channel_map,
)

In [7]:
print(frame_zstacks["images"].keys())
print(frame_zstacks["labels"].keys())

dict_keys(['D8-4_405', 'D8-4_488', 'D8-4_555', 'D8-4_640', 'D8-4_TRANS'])
dict_keys(['cell_mask', 'cytoplasm_mask', 'nuclei_mask', 'organoid_mask'])


In [8]:
images_data = []
labels_data = []
channel_names = []
label_names = []


for channel, stack in frame_zstacks["images"].items():
    dim = len(stack.shape)
    images_data.append(stack)
    channel_names.append(channel)

# Collect label data
if label_dir:
    for compartment_name, stack in frame_zstacks["labels"].items():
        if len(stack.shape) != dim:
            if len(stack.shape) == 3:
                stack = np.expand_dims(stack, axis=0)
        labels_data.append(stack)
        label_names.append(f"{compartment_name} (labels)")


# Stack the images and labels along a new axis for channels
images_data = np.stack(images_data, axis=0)
if labels_data:
    labels_data = np.stack(labels_data, axis=0)
    combined_data = np.concatenate((images_data, labels_data), axis=0)
    combined_channel_names = channel_names + label_names
else:
    combined_data = images_data
    combined_channel_names = channel_names
# Generate OME-XML metadata
ome_metadata = {
    "SizeC": combined_data.shape[0],
    "SizeZ": combined_data.shape[1],
    "SizeY": combined_data.shape[2],
    "SizeX": combined_data.shape[3],
    "PhysicalSizeX": scaling_values[2],
    "PhysicalSizeY": scaling_values[1],
    "PhysicalSizeZ": scaling_values[0],
    # note: we use 7-bit ascii compatible characters below
    # due to tifffile limitations
    "PhysicalSizeXUnit": "um",
    "PhysicalSizeYUnit": "um",
    "PhysicalSizeZUnit": "um",
    "Channel": [{"Name": name} for name in combined_channel_names],
}
ome_xml = generate_ome_xml(ome_metadata)
import tifffile as tiff

# Write the combined data to a single OME-TIFF
with tiff.TiffWriter(output_path, bigtiff=True) as tif:
    tif.write(combined_data, description=ome_xml, photometric="minisblack")

In [9]:
# import shutil
# shutil.rmtree(output_path, ignore_errors=True)
# nviz.image.tiff_to_ometiff(
#     image_dir=image_dir,
#     label_dir=label_dir,
#     output_path=output_path,
#     channel_map=channel_map,
#     scaling_values=scaling_values,
#     ignore=[],
# )

In [ ]:
viewer = view_ometiff_with_napari(
    ometiff_path=output_path,
    scaling_values=scaling_values,
    headless=False,
)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File ~/miniforge3/envs/viz_env/lib/python3.11/site-packages/napari/_qt/threads/status_checker.py:114, in StatusChecker.calculate_status(self=<napari._qt.threads.status_checker.StatusChecker object>)
    110     return
    112 try:
    113     # Calculate the status change from cursor's movement
--> 114     res = viewer._calc_status_from_cursor()
        viewer = Viewer(camera=Camera(center=(9.0, 76.90000000000002, 76.95000000000002), zoom=8.33333333333333, angles=(-113.7440292684523, 69.30619929735475, 1.3189468748748137), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(0.0, -3.196074331193339, 80.88795623513342, 78.34998716394749), scaled=True, style=<CursorStyle.STANDARD: 'standard'>, size=1.0), dims=Dims(ndim=4, ndisplay=3, order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3'), rollable=(True, True, True, Tr

In [11]:
if ANIMATE:
    mp4_file_dir = pathlib.Path(f"{animation_subparent_name}/mp4/").resolve()
    gif_file_dir = pathlib.Path(f"{animation_subparent_name}/gif/").resolve()
    mp4_file_dir.mkdir(parents=True, exist_ok=True)
    gif_file_dir.mkdir(parents=True, exist_ok=True)

    # make the viewer full screen
    viewer.window._qt_window.showMaximized()
    # hide the layer controls
    viewer.window._qt_viewer.dockLayerList.setVisible(False)
    # hide the layer controls
    viewer.window._qt_viewer.dockLayerControls.setVisible(False)

    # set the viewer to a set window size
    viewer.window._qt_window.resize(1000, 1000)
    # get the layer names in the viewer
    layer_names = [layer.name for layer in viewer.layers]
    # set all layers to not visible
    for layer_name in layer_names:
        print(f"Setting {layer_name} to not visible")
        viewer.layers[layer_name].visible = False
    for layer_name in layer_names:
        viewer.layers[layer_name].visible = True
        if ".tif" in layer_name:
            save_name = layer_name.split(".tif")[0]
        else:
            save_name = layer_name

        # map the layer name to the channel name
        if "Nuclei" in layer_name:
            save_name = "DNA"
        elif "Endoplasmic" in layer_name:
            save_name = "ER"
        elif "AGP" in layer_name:
            save_name = "AGP"
        elif "Mitochondria" in layer_name:
            save_name = "mitochondria"
        elif "Brightfield" in layer_name:
            save_name = "brightfield"
        else:
            save_name = layer_name

        save_path = pathlib.Path(f"{mp4_file_dir}/{well_fov}_{save_name}_animation.mp4")
        if "640" in layer_name:
            # increase contrast for the mitochondria
            viewer.layers[layer_name].contrast_limits = (0, 20000)
        animate_view(viewer, save_path, steps=30, easing="linear")
        viewer.layers[layer_name].visible = False
    print("All layers animated")
    # get all gifs in the directory
    mp4_file_path = list(pathlib.Path(mp4_file_dir).rglob("*.mp4"))
    for mp4_file in mp4_file_path:
        # change the path to the gif directory
        mp4_file = pathlib.Path(mp4_file)
        gif_file = pathlib.Path(gif_file_dir / f"{mp4_file.stem}.gif")
        mp4_file = str(mp4_file)
        gif_file = str(gif_file)
        mp4_to_gif(mp4_file, gif_file)

Setting D8-4_405 to not visible
Setting D8-4_488 to not visible
Setting D8-4_555 to not visible
Setting D8-4_640 to not visible
Setting D8-4_TRANS to not visible
Setting cell_mask (labels) to not visible
Setting cytoplasm_mask (labels) to not visible
Setting nuclei_mask (labels) to not visible
Setting organoid_mask (labels) to not visible
Rendering frames...


100%|██████████| 121/121 [00:03<00:00, 34.58it/s]


Rendering frames...


100%|██████████| 121/121 [00:03<00:00, 35.02it/s]


Rendering frames...


...

Rendering frames...


100%|██████████| 121/121 [00:03<00:00, 35.86it/s]


Rendering frames...


100%|██████████| 121/121 [00:03<00:00, 34.84it/s]


KeyboardInterrupt: 